In [1]:
import pandas as pd 
import numpy as np 
import pickle 
from scipy.stats import boxcox
from scipy.special import inv_boxcox
from statsmodels.formula.api import ols
pd.set_option('display.max_columns', None)

In [2]:
new_out=pd.read_parquet("/data/u_AA/throughput_opt/cells_D_70-80_new.parquet")
new_out.head(100)

,CELL_NAME,DATE_ID,DL_TRAFFIC_GB,DL_THROUGHPUT,traffic_1,traffic_2,traffic_3,traffic_4,traffic_5,traffic_6,traffic_7,traffic_8,traffic_9,traffic_10,traffic_11,traffic_12,traffic_13,traffic_14,traffic_15,traffic_16,traffic_17,traffic_18,traffic_19,traffic_20
0,LCAI4475_44751_N_H_D3_1,31-MAR-24 00.00.00,0.7718,12.4566,1.541702,2.447336,15.793541,19.973942,21.523539,19.833729,20.307869,17.571970,17.760643,16.308372,16.654806,15.134764,13.986072,10.460679,9.851224,8.922656,8.491942,11.763551,10.729980,13.461780
1,LCAI4475_44752_N_H_D3_2,31-MAR-24 00.00.00,0.8978,12.1087,-0.996446,-1.338298,9.493878,12.973226,16.186747,15.032557,13.238884,11.216076,9.790036,10.885660,10.662747,10.292619,8.429218,4.903831,4.294375,3.365808,2.935094,6.206705,5.173132,7.904929
2,LDEL4214_42141_N_H_D3_1,31-MAR-24 00.00.00,1.3600,23.7588,3.703723,2.112596,4.047024,6.237541,6.189240,4.619669,3.178203,0.787090,0.762873,-1.171283,-1.955864,-3.282435,-5.388680,-5.862129,-5.953356,-6.005208,-5.297835,-6.331406,-6.208432,-6.208432
3,LDEL4214_42142_N_H_D3_2,31-MAR-24 00.00.00,0.3885,28.8633,8.099332,8.479310,10.547331,19.417711,21.589378,20.250700,18.909824,18.449867,16.058754,15.807048,15.634503,14.272091,13.102354,9.579187,9.105740,9.014512,8.583798,8.952937,7.919365,7.988776
4,LDEL4214_42143_N_H_D3_3,31-MAR-24 00.00.00,0.6707,34.5562,14.532282,5.533092,7.015827,8.386817,10.938810,10.494510,8.567285,8.385531,8.517748,6.187579,5.356263,3.774854,2.612822,-1.138917,-1.230145,-1.230145,-2.130596,-0.540550,2.645386,2.768360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LDEL4486_44861_N_H_D3_1,28-MAR-24 00.00.00,0.1546,17.3784,7.713644,4.593584,3.199498,20.326933,20.373760,21.498379,19.270277,18.630470,17.501568,17.114252,15.563292,16.711399,15.407724,10.998979,10.392632,9.372838,9.320985,9.599614,11.559024,11.642785
96,LDEL4486_44862_N_H_D3_2,28-MAR-24 00.00.00,0.2943,16.3040,8.433776,4.147022,7.401944,19.947025,20.486877,20.932089,19.213659,18.470972,17.036493,17.117369,17.115025,15.938815,15.410843,10.705441,10.395751,9.375957,9.324103,12.595714,11.562142,11.532380
97,LDEL4486_44863_N_H_D3_3,28-MAR-24 00.00.00,4.4241,17.4483,20.904987,19.799992,18.251457,17.325937,15.802908,15.551201,15.507624,14.548403,14.177256,9.975138,9.162165,8.142368,8.090517,11.362127,10.328554,10.298792,10.298792,11.254774,11.254774,11.254774
98,LDEL4486_44864_N_H_D3_4,28-MAR-24 00.00.00,0.3923,38.6903,10.303645,10.118457,7.864703,16.150801,17.759882,16.582754,15.031740,14.691500,12.300390,12.048681,11.370137,10.428731,9.258986,5.581127,5.262377,5.171149,5.119297,5.488437,4.454864,4.524276


In [3]:
def negative_percentage(series):
    total_count = len(series)
    negative_count = (series < 0).sum()
    return (negative_count / total_count) * 100

In [12]:
negative = new_out.iloc[:,4:].apply(negative_percentage)

In [20]:
(new_out.iloc[:,4:] < 0).sum()

traffic_1     105
traffic_2     109
traffic_3      69
traffic_4      53
traffic_5      55
traffic_6      70
traffic_7      98
traffic_8     138
traffic_9     197
traffic_10    267
traffic_11    363
traffic_12    528
traffic_13    740
traffic_14    959
traffic_15    984
traffic_16    983
traffic_17    972
traffic_18    940
traffic_19    906
traffic_20    887
dtype: int64

In [18]:
print(negative)

traffic_1      2.204493
traffic_2      2.288474
traffic_3      1.448667
traffic_4      1.112744
traffic_5      1.154734
traffic_6      1.469662
traffic_7      2.057527
traffic_8      2.897334
traffic_9      4.136049
traffic_10     5.605711
traffic_11     7.621247
traffic_12    11.085450
traffic_13    15.536427
traffic_14    20.134369
traffic_15    20.659248
traffic_16    20.638253
traffic_17    20.407306
traffic_18    19.735461
traffic_19    19.021625
traffic_20    18.622717
dtype: float64


In [11]:
new_out.iloc[:,4:]

,traffic_1,traffic_2,traffic_3,traffic_4,traffic_5,traffic_6,traffic_7,traffic_8,traffic_9,traffic_10,traffic_11,traffic_12,traffic_13,traffic_14,traffic_15,traffic_16,traffic_17,traffic_18,traffic_19,traffic_20
0,1.541702,2.447336,15.793541,19.973942,21.523539,19.833729,20.307869,17.571970,17.760643,16.308372,16.654806,15.134764,13.986072,10.460679,9.851224,8.922656,8.491942,11.763551,10.729980,13.461780
1,-0.996446,-1.338298,9.493878,12.973226,16.186747,15.032557,13.238884,11.216076,9.790036,10.885660,10.662747,10.292619,8.429218,4.903831,4.294375,3.365808,2.935094,6.206705,5.173132,7.904929
2,3.703723,2.112596,4.047024,6.237541,6.189240,4.619669,3.178203,0.787090,0.762873,-1.171283,-1.955864,-3.282435,-5.388680,-5.862129,-5.953356,-6.005208,-5.297835,-6.331406,-6.208432,-6.208432
3,8.099332,8.479310,10.547331,19.417711,21.589378,20.250700,18.909824,18.449867,16.058754,15.807048,15.634503,14.272091,13.102354,9.579187,9.105740,9.014512,8.583798,8.952937,7.919365,7.988776
4,14.532282,5.533092,7.015827,8.386817,10.938810,10.494510,8.567285,8.385531,8.517748,6.187579,5.356263,3.774854,2.612822,-1.138917,-1.230145,-1.230145,-2.130596,-0.540550,2.645386,2.768360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4758,2.895465,3.926400,6.885109,6.072443,9.620099,9.000465,7.027365,4.421229,3.311231,4.540173,4.527991,2.989821,1.413976,-2.012548,-2.622001,-2.622001,-3.522451,-2.815079,-3.848650,-3.725675
4759,8.100406,8.491128,15.296582,20.276419,21.325222,20.678759,19.153418,16.461779,15.027788,15.318884,15.857121,14.252922,12.453064,9.755095,9.145640,9.145640,9.093788,9.462927,8.429356,8.498768
4760,3.520227,9.501584,18.939249,21.170681,20.151558,18.764187,18.193169,16.386196,16.134487,15.826531,14.420023,13.250281,9.563361,9.253672,9.162443,8.731731,12.003338,10.969767,11.039179,11.039179
4761,17.286821,19.913870,18.374693,17.742065,15.700374,14.303277,13.949365,13.950676,12.714054,10.853208,7.825519,7.734292,7.734292,7.682439,8.051578,7.018005,7.087420,8.043397,8.043397,8.043397


In [22]:
new_out2=pd.read_parquet("/data/u_AA/throughput_opt/cells_D_80-90_new.parquet")
new_out2.head(100)

,CELL_NAME,DATE_ID,DL_TRAFFIC_GB,DL_THROUGHPUT,traffic_1,traffic_2,traffic_3,traffic_4,traffic_5,traffic_6,traffic_7,traffic_8,traffic_9,traffic_10,traffic_11,traffic_12,traffic_13,traffic_14,traffic_15,traffic_16,traffic_17,traffic_18,traffic_19,traffic_20
0,LCAI4021_40213_N_H_D6_3,31-MAR-24 00.00.00,4.7646,12.4225,26.986233,26.914394,23.210812,22.830812,21.109755,20.381077,20.361658,18.765589,18.601402,18.497355,16.134016,14.675576,11.648430,11.213098,11.213098,13.367520,13.367520,14.323495,14.236478,14.026202
1,LCAI4210_42103_N_H_D6_3,31-MAR-24 00.00.00,3.6770,12.4813,19.128511,20.300550,20.753080,18.218349,16.589525,16.280153,13.870978,14.120371,14.079290,16.337782,12.800010,9.842729,8.436767,5.530853,5.018307,9.688774,11.843197,11.843197,12.799172,12.712155
2,LCAI2880_28803_N_H_D6_3,31-MAR-24 00.00.00,3.2343,12.5881,17.615522,18.241083,18.593906,16.292580,14.281680,14.624330,13.246308,12.768367,12.450427,11.463611,10.606977,8.492419,7.023645,5.965417,3.218671,7.889134,9.917700,10.043556,10.999536,10.912515
3,LCAI3275_61978_V_H_D6_3,31-MAR-24 00.00.00,3.3968,12.9628,20.206539,21.862404,20.853022,19.160603,15.602343,15.314862,13.562346,16.311598,14.112458,13.458225,12.276201,10.590116,9.258115,7.800612,5.367687,5.367687,7.535961,7.661817,8.617795,8.617795
4,LCAI3357_33571_N_H_D6_1,31-MAR-24 00.00.00,7.0325,14.7591,10.110154,11.222150,9.646001,9.650051,8.972165,8.727252,6.881879,5.839895,4.028239,2.490679,-0.388027,2.247449,4.431471,4.943262,5.899243,5.812224,5.601949,5.601949,2.885767,2.885767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LUPP5749_57492_N_H_D6_2,28-MAR-24 00.00.00,1.0470,19.5364,2.970102,2.573769,19.278755,21.393631,20.182419,19.188433,18.501673,19.936800,18.585052,18.116205,17.640804,17.259073,15.797351,14.345350,13.522406,12.296916,8.910174,8.739374,10.921998,10.934331
96,LUPP5749_57494_N_H_D6_4,28-MAR-24 00.00.00,4.2827,16.6169,19.517954,20.029959,17.891146,15.801400,14.710017,14.482805,15.484532,13.285399,12.631165,11.774530,10.008753,8.607658,7.261872,4.611623,4.611623,9.442280,9.568136,10.524113,10.437096,10.226820
97,LUPP5756_57561_N_H_D6_1,28-MAR-24 00.00.00,6.1127,20.0062,6.134304,5.084445,5.552083,4.174061,3.804314,3.510300,2.523488,1.420017,-0.345758,-0.922608,-2.872761,-5.645085,-5.619508,-3.436884,-2.895262,-1.939284,-2.026303,-2.236577,-2.236577,-2.236577
98,LUPP5756_57562_N_H_D6_2,28-MAR-24 00.00.00,3.7584,15.2661,25.081467,25.649128,25.479340,21.991976,22.028135,20.333357,19.865959,19.921749,18.325680,18.161493,17.313946,15.745885,14.287439,11.069239,13.536378,13.536378,15.690799,15.690799,16.646774,16.559757


In [23]:
negative = new_out2.iloc[:,4:].apply(negative_percentage)
negative

traffic_1     2.665513
traffic_2     1.548986
traffic_3     0.428526
traffic_4     0.231955
traffic_5     0.263406
traffic_6     0.318446
traffic_7     0.558264
traffic_8     0.888504
traffic_9     1.434974
traffic_10    2.083661
traffic_11    3.101903
traffic_12    4.124076
traffic_13    5.531530
traffic_14    7.025476
traffic_15    8.130209
traffic_16    8.774965
traffic_17    8.955811
traffic_18    8.723856
traffic_19    8.083032
traffic_20    7.560151
dtype: float64

In [24]:
new_out3=pd.read_parquet("/data/u_AA/throughput_opt/cells_D_90-100_new.parquet")
new_out3.head(100)

,CELL_NAME,DATE_ID,DL_TRAFFIC_GB,DL_THROUGHPUT,traffic_1,traffic_2,traffic_3,traffic_4,traffic_5,traffic_6,traffic_7,traffic_8,traffic_9,traffic_10,traffic_11,traffic_12,traffic_13,traffic_14,traffic_15,traffic_16,traffic_17,traffic_18,traffic_19,traffic_20
0,LCAI1994_19942_N_H_D6_2,29-MAR-24 00.00.00,5.9426,21.9594,28.225882,25.817459,27.083445,24.710949,23.677502,23.587803,22.140566,20.195568,18.870953,15.102797,14.576957,14.784098,17.686573,17.545464,19.881531,20.750488,20.540213,20.540213,20.540213,20.540213
1,LCAI1994_19943_N_H_D6_3,29-MAR-24 00.00.00,1.7253,15.6750,11.955009,25.018255,25.762383,28.295443,27.207451,25.150581,24.624014,24.225857,21.903894,21.901360,20.219509,19.627922,18.476685,15.655602,13.233616,13.440758,16.343224,16.202114,18.538187,18.451168
2,LCAI2006_20061_N_H_D6_1,29-MAR-24 00.00.00,6.2157,12.1941,21.162437,21.008007,21.186728,18.309677,19.559685,19.061165,17.885931,15.586812,14.103106,10.446945,10.589156,13.491625,18.020985,20.190659,20.316515,21.185472,20.975197,20.975197,20.975197,18.259018
3,LCAI2011_20111_N_H_D6_1,29-MAR-24 00.00.00,4.6236,15.3835,27.077324,27.345066,22.855453,20.657557,18.930386,17.996145,18.087433,17.416626,15.813995,14.543307,12.009780,9.587793,9.794935,9.794935,9.653824,12.089065,12.002048,12.747751,12.747751,12.747751
4,LCAI2011_20112_N_H_D6_2,29-MAR-24 00.00.00,7.7082,15.3746,21.439562,21.103914,18.940145,18.862402,18.191595,16.588964,15.156036,12.622508,10.200521,10.407663,10.407663,10.266552,12.701793,12.614776,13.360479,13.360479,13.360479,13.360479,13.360479,13.360479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LCAI4481_44811_N_H_D6_1,31-MAR-24 00.00.00,1.6682,23.2823,13.586741,25.823143,27.528223,28.815548,28.511915,24.844366,25.689320,25.326824,23.450613,23.448076,22.777269,21.174639,19.955729,17.250780,14.541426,14.748566,17.651039,17.509930,19.945171,19.945171
96,LCAI4481_44812_N_H_D6_2,31-MAR-24 00.00.00,1.7562,17.7214,13.379511,26.447695,28.948980,29.553675,28.606018,26.285917,25.932846,24.184942,23.694139,23.691601,22.009750,21.418167,19.307327,17.494305,14.784946,14.992087,17.894566,17.753456,20.188698,20.188698
97,LCAI4492_44922_N_H_D7_2,31-MAR-24 00.00.00,2.8158,34.9067,9.453126,10.645544,9.685101,10.246726,8.129984,6.813030,4.819957,3.842367,3.888363,2.595509,0.671386,-0.663750,-2.879183,-5.083076,-5.083076,-4.804446,-4.804446,-4.193640,-3.324680,-3.534956
98,LCAI4492_44923_N_H_D7_3,31-MAR-24 00.00.00,4.0620,13.6169,24.582319,25.705143,22.999775,21.850685,19.468496,17.352631,16.137362,15.818930,14.707330,12.665121,11.221050,7.163098,6.858881,6.858881,7.137511,7.221272,7.206923,8.075881,7.865608,7.865608


In [26]:
negative = new_out3.iloc[:,4:].apply(negative_percentage)
negative

traffic_1      0.109148
traffic_2      0.166564
traffic_3      0.197831
traffic_4      0.292198
traffic_5      0.426359
traffic_6      0.631580
traffic_7      0.930600
traffic_8      1.292153
traffic_9      1.786161
traffic_10     2.427405
traffic_11     3.213043
traffic_12     4.258476
traffic_13     5.505150
traffic_14     6.973532
traffic_15     8.736385
traffic_16    10.300271
traffic_17     9.678355
traffic_18     8.685222
traffic_19     7.948473
traffic_20     7.402733
dtype: float64

In [27]:
new_out4=pd.read_parquet("/data/u_AA/throughput_opt/cells_T4_90-100_new.parquet")
new_out4.head(100)

,CELL_NAME,DATE_ID,DL_TRAFFIC_GB,DL_THROUGHPUT,traffic_1,traffic_2,traffic_3,traffic_4,traffic_5,traffic_6,traffic_7,traffic_8,traffic_9,traffic_10,traffic_11,traffic_12,traffic_13,traffic_14,traffic_15,traffic_16,traffic_17,traffic_18,traffic_19,traffic_20
0,LCAI2026_36901_N_H_T4_1,29-MAR-24 00.00.00,8.1559,14.5630,21.940550,21.178782,20.035883,19.622025,18.400330,14.585699,13.097974,13.006745,11.973176,13.735162,16.379944,16.502916,16.502916,17.458895,17.458895,17.458895,17.458895,14.742713,14.742713,14.742713
1,LCAI1974_33881_N_H_T4_1,29-MAR-24 00.00.00,3.3169,18.3096,26.239807,26.400797,26.924866,24.947567,25.110594,22.208862,21.819218,22.800043,20.768229,19.572206,15.593820,15.371647,15.280416,14.246842,14.525472,14.525472,14.594886,14.594886,15.550867,15.550867
2,LCAI1974_33884_N_H_T4_4,29-MAR-24 00.00.00,12.8695,27.4779,18.805101,14.733291,14.120473,13.086902,13.086902,14.848889,19.068396,19.191370,20.147350,20.147350,20.147350,20.147350,20.147350,20.147350,20.147350,23.697515,25.812777,25.725756,26.079813,30.847281
3,LCAI1982_39093_N_H_T4_3,29-MAR-24 00.00.00,4.5216,13.1303,23.397552,23.505079,22.589235,22.154533,19.207426,18.792772,19.527412,18.509056,16.561947,13.543185,13.112475,13.021247,11.987675,12.266304,12.266304,12.335715,12.335715,13.291697,13.291697,13.291697
4,LCAI2076_60951_N_H_T4_1,29-MAR-24 00.00.00,0.8036,26.7061,27.400173,25.949251,25.056683,28.332531,29.086620,27.747469,26.284842,25.269630,21.622269,22.189623,21.769476,20.378670,18.268387,16.806208,16.284269,15.250696,15.250696,15.257998,18.174107,18.297081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LDEL4540_55402_N_H_T4_2,31-MAR-24 00.00.00,1.4420,31.9878,24.121922,22.157406,24.061165,24.619673,25.788122,23.110792,20.854931,18.575153,18.745853,18.061771,17.605776,15.064727,13.099712,12.471745,12.380518,11.346947,11.625576,14.270356,14.393332,14.393332
96,LDEL4540_55403_N_H_T4_3,31-MAR-24 00.00.00,0.4751,40.8682,25.442474,23.456686,22.195185,23.505062,23.651213,23.677011,22.382032,20.917404,18.939491,18.860474,18.262049,17.287783,15.340677,13.427438,12.996727,12.905501,11.871934,12.150563,12.150563,12.273537
97,LCAI3438_61521_N_H_T4_1,31-MAR-24 00.00.00,4.1715,15.4255,27.232927,27.446661,25.402557,24.566801,23.341883,22.352623,22.019464,21.315567,20.119549,16.304918,16.297844,16.206617,15.173049,15.451678,15.451678,15.574655,15.574655,16.530630,16.530630,16.530630
98,LCAI3438_61523_N_H_T4_3,31-MAR-24 00.00.00,3.8769,12.5038,27.169439,29.689888,28.562410,26.984114,24.899035,22.794785,23.284462,23.261774,21.918318,19.539244,17.256540,16.734602,15.701033,15.701033,15.708332,18.624445,18.747421,19.703400,19.703400,19.703400


In [28]:
new_out5=pd.read_parquet("/data/u_AA/throughput_opt/cells_T8_90-100_new.parquet")
new_out5.head(100)

,CELL_NAME,DATE_ID,DL_TRAFFIC_GB,DL_THROUGHPUT,traffic_1,traffic_2,traffic_3,traffic_4,traffic_5,traffic_6,traffic_7,traffic_8,traffic_9,traffic_10,traffic_11,traffic_12,traffic_13,traffic_14,traffic_15,traffic_16,traffic_17,traffic_18,traffic_19,traffic_20
0,LCAI2011_39476_N_H_T8_1,29-MAR-24 00.00.00,2.7634,20.6368,23.514854,28.105865,27.574295,26.453737,26.861393,25.675787,24.934923,23.004051,23.666626,21.876120,20.284063,17.544882,16.253225,16.253225,16.460367,16.460367,16.460367,18.755899,18.755899,19.711878
1,LCAI2011_39477_N_H_T8_2,29-MAR-24 00.00.00,7.1722,13.5447,13.989623,12.990799,11.777420,10.913589,10.716854,8.976933,5.618886,5.386978,4.262182,4.197992,4.197992,7.114103,9.825170,9.951027,10.907006,10.907006,10.907006,10.765897,8.049716,8.049716
2,LCAI2054_44039_N_H_T8_4,29-MAR-24 00.00.00,4.7807,12.5028,26.471096,25.400042,25.820936,24.591063,23.686907,21.961079,22.529940,20.739433,19.878300,16.504692,15.213037,15.213037,15.420177,15.420177,15.420177,17.855425,17.855425,18.811401,18.811401,18.670292
3,LCAI1956_63558_N_H_T8_3,29-MAR-24 00.00.00,8.6807,14.0903,18.261621,16.330748,17.024485,15.372025,13.833935,11.043334,9.751681,9.751681,9.958821,9.958821,9.958821,12.394064,12.394064,13.350037,13.350037,13.208929,13.208929,13.208929,13.208929,13.121913
4,LCAI1969_65477_N_H_T8_2,29-MAR-24 00.00.00,2.5065,24.3676,16.875288,19.132236,19.294746,19.098360,17.993126,18.106136,15.492725,15.650108,15.861083,15.043838,13.614956,11.307610,10.107183,10.015956,9.951766,9.951766,12.867879,15.704801,15.704801,16.660778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,LCAI2695_63628_N_H_T8_3,31-MAR-24 00.00.00,1.7649,12.9769,17.248756,27.059948,31.493944,30.743732,29.765139,30.186033,28.740000,27.936464,26.503813,27.124268,25.333761,23.804029,21.506557,20.214903,20.214903,20.422045,20.422045,20.422045,22.857285,22.857285
96,LCAI2898_62846_N_H_T8_1,31-MAR-24 00.00.00,0.9803,17.1874,15.343355,13.021963,17.994223,20.598373,21.183170,19.737103,19.769085,18.009090,17.171150,16.750063,16.999733,15.691748,13.331366,11.306373,10.181573,10.181573,10.388715,10.388715,10.388715,12.823956
97,LCAI2898_62847_N_H_T8_2,31-MAR-24 00.00.00,0.2645,15.1922,16.659901,14.595110,14.684896,27.080658,26.597692,26.042301,24.901222,25.178915,23.190130,23.416271,23.883837,22.042809,20.328560,16.903255,16.835102,15.710301,15.917443,15.917443,15.917443,18.226831
98,LCAI2898_62849_N_H_T8_4,31-MAR-24 00.00.00,0.6006,13.3357,11.921940,12.666064,20.178244,22.887264,23.559406,23.453058,21.908106,22.080366,19.733267,19.741552,20.310415,18.323822,17.642418,14.086431,12.886003,12.794777,13.001919,13.001919,13.001919,15.437155


In [ ]:
explainer = shap.Explainer(model.predict, X_test)